In [3]:
# ====================================================================
# CELDA 1: CARGA Y AUDITORÍA DE ARTEFACTOS DEL PROYECTO
# ====================================================================
import pandas as pd
import numpy as np
import json
import joblib
import warnings

warnings.filterwarnings("ignore")

print("--- [INICIO] Cargando y Auditando Artefactos del Proyecto ---")

# --- 1. Cargar los tres artefactos clave ---
try:
    # Artefacto 1: El DataFrame con todas las características y el target
    data_path = 'dataframes/data_with_target.parquet'
    df_completo = pd.read_parquet(data_path)
    print(f"✅ [1/3] DataFrame completo cargado desde '{data_path}' (Shape: {df_completo.shape})")

    # Artefacto 2: La lista de características importantes seleccionadas por RFECV
    features_path = 'important_features.json'
    with open(features_path, 'r') as f:
        important_feature_names = json.load(f)
    print(f"✅ [2/3] Lista de características importantes cargada desde '{features_path}' ({len(important_feature_names)} features)")

    # Artefacto 3: El paquete de estrategia con los modelos entrenados
    package_path = 'models/final_strategy_package.joblib'
    strategy_package = joblib.load(package_path)
    print(f"✅ [3/3] Paquete de modelos cargado desde '{package_path}'")

except FileNotFoundError as e:
    print(f"❌ ERROR: No se pudo encontrar un archivo esencial: {e}")
    print("   -> Asegúrate de haber ejecutado los notebooks 01 y 02 completamente y de que los archivos existen.")
except Exception as e:
    print(f"❌ ERROR inesperado durante la carga: {e}")


# --- 2. Auditoría del Estado Cargado ---
print("\n" + "="*25 + " AUDITORÍA DEL ESTADO DEL PROYECTO " + "="*25)

# A. Consistencia de las Características
print("\n--- A. Consistencia de Características ---")
original_cols_to_keep = [col.split('__')[1] for col in important_feature_names]
print(f"Las {len(original_cols_to_keep)} características que nuestro modelo utiliza son:")
print(original_cols_to_keep)

# B. Auditoría del Pipeline Guardado
print("\n--- B. Auditoría del Pipeline Guardado ---")
loaded_pipeline = strategy_package.get('primary_model_pipeline')
if loaded_pipeline:
    print("Pipeline Primario encontrado en el paquete:")
    print(loaded_pipeline)
else:
    print("❌ No se encontró 'primary_model_pipeline' en el paquete guardado.")

# C. Creación del DataFrame Final para Backtesting (La "Verdad Absoluta")
print("\n--- C. Construcción del DataFrame Final de Trabajo ---")
structural_cols = ['ticker', 'timestamp', 'open', 'high', 'low', 'close', 'volume']
final_cols_to_keep = list(set(original_cols_to_keep + structural_cols))
X_final_auditado = df_completo[final_cols_to_keep].copy()
y_final_auditado = df_completo['target'].copy()

print("A partir de ahora, trabajaremos con estos DataFrames auditados:")
print(f" -> X_final_auditado (Shape: {X_final_auditado.shape})")
print(f" -> y_final_auditado (Shape: {y_final_auditado.shape})")


print("\n--- [FIN] Auditoría completada. El estado base del proyecto ha sido cargado en memoria. ---")

--- [INICIO] Cargando y Auditando Artefactos del Proyecto ---
✅ [1/3] DataFrame completo cargado desde 'dataframes/data_with_target.parquet' (Shape: (5391, 45))
✅ [2/3] Lista de características importantes cargada desde 'important_features.json' (15 features)
✅ [3/3] Paquete de modelos cargado desde 'models/final_strategy_package.joblib'

========================= AUDITORÍA DEL ESTADO DEL PROYECTO =========================

--- A. Consistencia de Características ---
Las 15 características que nuestro modelo utiliza son:
['close', 'macd_signal', 'macd_hist', 'funding_rate', 'spy_close', 'vix_close', 'tnx_close', 'dxy_close', 'gc_close', 'cl_close', 'log_return', 'volatility_7d', 'price_to_ema_ratio', 'macd_norm', 'log_return_gc_close']

--- B. Auditoría del Pipeline Guardado ---
Pipeline Primario encontrado en el paquete:
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['close',

In [4]:
# ====================================================================
# CELDA 2: PRUEBA DE FUEGO CORREGIDA (LÓGICA DE ENTRENAMIENTO AISLADA)
# ====================================================================
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

print("--- [INICIO] Prueba de Fuego para el Primer Split de BTC-USD (CORREGIDA) ---")

# --- 1. PREPARACIÓN ROBUSTA DE DATOS ---
ticker_de_prueba = 'BTC-USD'

# ✅ PASO 1: Verificar la estructura de los datos
print(f"📊 Diagnóstico inicial:")
print(f"   - X_final_auditado shape: {X_final_auditado.shape}")
print(f"   - y_final_auditado shape: {y_final_auditado.shape}")
print(f"   - X_final_auditado index: {type(X_final_auditado.index)}")
print(f"   - y_final_auditado index: {type(y_final_auditado.index)}")

# ✅ PASO 2: Sincronización correcta de índices
# Asegurarnos de que timestamp sea el índice en ambos DataFrames
if 'timestamp' in X_final_auditado.columns:
    X_final_auditado = X_final_auditado.set_index('timestamp')

# Si y_final_auditado no tiene timestamp como índice, necesitamos alinearlo correctamente
if not isinstance(y_final_auditado.index, pd.DatetimeIndex):
    # Asumir que y_final_auditado debe tener el mismo índice que X_final_auditado
    # pero solo para las filas que realmente existen
    common_index = X_final_auditado.index
    if len(y_final_auditado) == len(common_index):
        y_final_auditado.index = common_index
    else:
        raise ValueError(f"Incompatibilidad de datos: X tiene {len(X_final_auditado)} filas, "
                        f"pero y tiene {len(y_final_auditado)} filas. "
                        f"Los datos deben estar perfectamente alineados.")

# ✅ PASO 3: Filtrado por ticker con verificación CORRECTA
ticker_mask = X_final_auditado['ticker'] == ticker_de_prueba
ticker_X = X_final_auditado[ticker_mask].drop(columns=['ticker'])

print(f"🔍 Filtrado por ticker '{ticker_de_prueba}':")
print(f"   - Filas encontradas: {ticker_mask.sum()}")
print(f"   - ticker_X shape: {ticker_X.shape}")

# ⚠️ CLAVE: Filtrar y_final_auditado usando la MISMA MÁSCARA
# No usar índices porque pueden estar duplicados entre tickers
ticker_y = y_final_auditado[ticker_mask]

print(f"   - ticker_y shape después del filtrado: {ticker_y.shape}")

# Verificar que ambos tienen exactamente las mismas filas
if len(ticker_X) != len(ticker_y):
    print(f"❌ ERROR: Después del filtrado por máscara:")
    print(f"   - ticker_X: {len(ticker_X)} filas")
    print(f"   - ticker_y: {len(ticker_y)} filas")
    
    # Diagnóstico adicional
    print(f"📊 Análisis de índices:")
    print(f"   - X_final_auditado index duplicates: {X_final_auditado.index.duplicated().sum()}")
    print(f"   - y_final_auditado index duplicates: {y_final_auditado.index.duplicated().sum()}")
    
    raise ValueError(f"El filtrado por máscara no funcionó correctamente. "
                    f"Esto indica que X_final_auditado e y_final_auditado no están "
                    f"perfectamente alineados fila por fila.")

# ✅ PASO 4: Verificación final de alineación
assert len(ticker_X) == len(ticker_y), f"Desalineación: X={len(ticker_X)}, y={len(ticker_y)}"
assert (ticker_X.index == ticker_y.index).all(), "Los índices no coinciden exactamente"

print(f"✅ Datos sincronizados correctamente:")
print(f"   - ticker_X shape: {ticker_X.shape}")
print(f"   - ticker_y shape: {ticker_y.shape}")
print(f"   - Índices alineados: {(ticker_X.index == ticker_y.index).all()}")

# ✅ PASO 5: Creación de splits con verificación
train_period = 189
test_period = 63

if len(ticker_X) < train_period + test_period:
    raise ValueError(f"Datos insuficientes: se necesitan {train_period + test_period} filas, "
                    f"pero solo hay {len(ticker_X)} disponibles")

train_X_split_1 = ticker_X.iloc[0:train_period]
train_y_split_1 = ticker_y.iloc[0:train_period]
test_X_split_1 = ticker_X.iloc[train_period:train_period + test_period]

print(f"✅ Splits creados correctamente:")
print(f"   - train_X_split_1: {train_X_split_1.shape}")
print(f"   - train_y_split_1: {train_y_split_1.shape}")
print(f"   - test_X_split_1: {test_X_split_1.shape}")

# --- 2. FUNCIÓN DE ENTRENAMIENTO ROBUSTA ---
def train_and_predict_on_split(train_X, train_y, test_X):
    """
    Función que replica la arquitectura auditada para un solo split.
    Con verificaciones adicionales de robustez.
    """
    # ✅ Verificación inicial de alineación
    assert len(train_X) == len(train_y), f"Train desalineado: X={len(train_X)}, y={len(train_y)}"
    assert (train_X.index == train_y.index).all(), "Índices de train no coinciden"
    
    print(f"   📊 Input data: train_X={train_X.shape}, train_y={train_y.shape}, test_X={test_X.shape}")
    
    # a. Filtrado de datos (solo para entrenamiento)
    valid_mask = train_y != 0
    valid_index = train_y[valid_mask].index
    train_X_filtered = train_X.loc[valid_index]
    train_y_binary = train_y.loc[valid_index].map({-1: 0, 1: 1})
    
    if len(train_X_filtered) < 50:
        print(f"   ⚠️ Datos insuficientes tras filtrar: {len(train_X_filtered)} < 50")
        return None, None
    
    print(f"   ✅ Datos filtrados: {train_X_filtered.shape[0]} filas válidas de {len(train_X)} originales")
    print(f"   📊 Distribución de clases: {train_y_binary.value_counts().to_dict()}")
    
    # b. Construcción del Pipeline
    numeric_features = train_X_filtered.columns.tolist()
    
    preprocessor = ColumnTransformer(
        transformers=[('num', StandardScaler(), numeric_features)],
        remainder='passthrough'
    )
    
    primary_model_wf = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('classifier', LGBMClassifier(
            class_weight='balanced', 
            objective='binary', 
            random_state=42, 
            verbosity=-1
        ))
    ])
    
    print(f"   ✅ Pipeline construido con {len(numeric_features)} características")
    
    # c. Entrenamiento del modelo primario
    try:
        primary_model_wf.fit(train_X_filtered, train_y_binary)
        print(f"   ✅ Modelo primario entrenado exitosamente")
    except Exception as e:
        print(f"   ❌ Error en entrenamiento: {e}")
        raise
    
    # d. Meta-Modelo y Predicción
    primary_train_proba = primary_model_wf.predict_proba(train_X_filtered)
    X_meta_train = pd.DataFrame({'p': primary_train_proba.max(axis=1)})
    meta_model = LogisticRegression().fit(X_meta_train, train_y_binary)
    
    primary_train_preds = np.argmax(primary_train_proba, axis=1)
    meta_train_probs = meta_model.predict_proba(X_meta_train)[:, 1]
    
    # Optimización de threshold
    thresholds = np.arange(0.50, 0.70, 0.02)
    best_f1 = -1
    optimal_threshold = 0.55
    
    for thresh in thresholds:
        mask = meta_train_probs >= thresh
        if mask.sum() > 5:
            f1 = f1_score(train_y_binary.iloc[mask], primary_train_preds[mask], zero_division=0)
            if f1 > best_f1:
                best_f1, optimal_threshold = f1, thresh
    
    print(f"   📈 Threshold óptimo: {optimal_threshold:.3f} (F1={best_f1:.3f})")
    
    # Predicción en test
    primary_test_proba = primary_model_wf.predict_proba(test_X)
    primary_test_preds = np.argmax(primary_test_proba, axis=1)
    meta_test_probs = meta_model.predict_proba(
        pd.DataFrame({'p': primary_test_proba.max(axis=1)})
    )[:, 1]
    
    entries = (meta_test_probs >= optimal_threshold)
    buy_signals = pd.Series((primary_test_preds == 1) & entries, index=test_X.index)
    sell_signals = pd.Series((primary_test_preds == 0) & entries, index=test_X.index)
    
    print(f"   🎯 Señales generadas: {buy_signals.sum()} compras, {sell_signals.sum()} ventas")
    
    return buy_signals, sell_signals

# --- 3. PRUEBA DE FUEGO ---
print(f"\n🔥 EJECUTANDO PRUEBA DE FUEGO...")
try:
    buy_s, sell_s = train_and_predict_on_split(train_X_split_1, train_y_split_1, test_X_split_1)
    
    print(f"\n{'='*60}")
    print(f"🎉 RESULTADO: PRUEBA SUPERADA")
    print(f"{'='*60}")
    
    if buy_s is not None:
        print(f"✅ La función se ejecutó sin errores")
        print(f"📊 Señales generadas en el primer split:")
        print(f"   - Compras: {buy_s.sum()}")
        print(f"   - Ventas: {sell_s.sum()}")
        print(f"   - Total entradas: {(buy_s | sell_s).sum()}")
        print(f"   - Cobertura: {(buy_s | sell_s).sum() / len(buy_s) * 100:.1f}%")
    else:
        print(f"✅ La función se ejecutó correctamente")
        print(f"⚠️ No se generaron señales (datos insuficientes tras filtrar)")

except Exception as e:
    print(f"\n{'='*60}")
    print(f"❌ RESULTADO: PRUEBA FALLIDA")
    print(f"{'='*60}")
    print(f"Error encontrado: {str(e)}")
    print(f"\n📋 Traceback completo:")
    import traceback
    traceback.print_exc()

print(f"\n🏁 Prueba de fuego completada.")

--- [INICIO] Prueba de Fuego para el Primer Split de BTC-USD (CORREGIDA) ---
📊 Diagnóstico inicial:
   - X_final_auditado shape: (5391, 21)
   - y_final_auditado shape: (5391,)
   - X_final_auditado index: <class 'pandas.core.indexes.range.RangeIndex'>
   - y_final_auditado index: <class 'pandas.core.indexes.range.RangeIndex'>
🔍 Filtrado por ticker 'BTC-USD':
   - Filas encontradas: 459
   - ticker_X shape: (459, 19)
   - ticker_y shape después del filtrado: (459,)
✅ Datos sincronizados correctamente:
   - ticker_X shape: (459, 19)
   - ticker_y shape: (459,)
   - Índices alineados: True
✅ Splits creados correctamente:
   - train_X_split_1: (189, 19)
   - train_y_split_1: (189,)
   - test_X_split_1: (63, 19)

🔥 EJECUTANDO PRUEBA DE FUEGO...
   📊 Input data: train_X=(189, 19), train_y=(189,), test_X=(63, 19)
   ✅ Datos filtrados: 182 filas válidas de 189 originales
   📊 Distribución de clases: {0: 95, 1: 87}
   ✅ Pipeline construido con 19 características
   ✅ Modelo primario entrenado 

In [5]:
# ====================================================================
# PASO 17: MÓDULO DE WALK-FORWARD ANALYSIS FINAL (BASADO EN CELDA 2)
# ====================================================================
import vectorbt as vbt
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np

print("--- [INICIO] Ejecutando el Walk-Forward Analysis completo ---")

# --- 1. PREPARACIÓN ---
# Usamos las variables auditadas de la CELDA 1 y la función de la CELDA 2
all_tickers = X_final_auditado['ticker'].unique()
all_stats = []

# --- 2. BUCLE PRINCIPAL POR CADA TICKER ---
for ticker in all_tickers:
    print(f"\n{'='*25} PROCESANDO TICKER: {ticker} {'='*25}")
    
    # --- PREPARACIÓN DE DATOS (LÓGICA EXACTA DE TU CELDA 2) ---
    ticker_mask = X_final_auditado['ticker'] == ticker
    ticker_X = X_final_auditado[ticker_mask].drop(columns=['ticker'])
    ticker_y = y_final_auditado[ticker_mask]

    # Verificación
    assert len(ticker_X) == len(ticker_y), "Error de alineación inicial"
    assert (ticker_X.index == ticker_y.index).all(), "Error de índice inicial"
    # --- FIN DE LA LÓGICA COPIADA ---

    train_period = 189
    test_period = 63
    
    if len(ticker_X) < train_period + test_period:
        print(f" -> Datos insuficientes. Saltando...")
        continue
    
    n_splits = (len(ticker_X) - train_period) // test_period
    if n_splits < 1:
        print(f" -> No hay suficientes datos para un split. Saltando...")
        continue

    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_period)
    
    all_buy_signals = pd.Series(dtype=bool)
    all_sell_signals = pd.Series(dtype=bool)
    
    print(f" -> Ejecutando {n_splits} splits para {ticker}...")
    
    # --- Bucle Interno por cada Split ---
    for train_index, test_index in tscv.split(ticker_X):
        split_train_X, split_test_X = ticker_X.iloc[train_index], ticker_X.iloc[test_index]
        split_train_y, _ = ticker_y.iloc[train_index], ticker_y.iloc[test_index]
        
        # Ejecutamos nuestra función ya validada 'train_and_predict_on_split'
        buy_s, sell_s = train_and_predict_on_split(split_train_X, split_train_y, split_test_X)
        
        if buy_s is not None:
            all_buy_signals = pd.concat([all_buy_signals, buy_s])
            all_sell_signals = pd.concat([all_sell_signals, sell_s])

    # --- 3. BACKTEST PARA EL TICKER ACTUAL ---
    if all_buy_signals.empty or all_buy_signals.sum() == 0:
        print(f" -> No se generaron operaciones para {ticker} en todo el análisis.")
        continue

    all_buy_signals = all_buy_signals[~all_buy_signals.index.duplicated(keep='first')]
    all_sell_signals = all_sell_signals.loc[all_buy_signals.index]
    
    # El precio 'close' está dentro de ticker_X
    price_data_for_pf = ticker_X
    
    valid_indices = all_buy_signals.index.intersection(price_data_for_pf.index)
    if valid_indices.empty: continue

    wf_portfolio = vbt.Portfolio.from_signals(
        close=price_data_for_pf.loc[valid_indices, 'close'],
        entries=all_buy_signals.reindex(valid_indices, fill_value=False),
        exits=all_sell_signals.reindex(valid_indices, fill_value=False),
        fees=0.002, sl_stop=0.05, tp_stop=0.05, init_cash=100000, freq='D'
    )
    
    ticker_stats = wf_portfolio.stats()
    ticker_stats.name = ticker
    all_stats.append(ticker_stats)
    print(f" ✔️ RESULTADOS PARA {ticker}: Total Return: {ticker_stats['Total Return [%]']:.2f}%, Win Rate: {ticker_stats['Win Rate [%]']:.2f}%, Trades: {ticker_stats['Total Trades']}")

# --- 4. INFORME FINAL AGREGADO ---
print(f"\n{'='*25} INFORME AGREGADO FINAL DEL WALK-FORWARD {'='*25}")
if not all_stats:
    print("No se generaron estadísticas para ningún ticker.")
else:
    final_stats_df = pd.DataFrame(all_stats)
    print("Rendimiento 'Out-of-Sample' de la estrategia por activo:")
    print(final_stats_df[['Total Return [%]', 'Max Drawdown [%]', 'Win Rate [%]', 'Total Trades', 'Sharpe Ratio', 'Sortino Ratio']])

--- [INICIO] Ejecutando el Walk-Forward Analysis completo ---

========================= PROCESANDO TICKER: ADA-USD =========================
 -> Ejecutando 4 splits para ADA-USD...
   📊 Input data: train_X=(207, 19), train_y=(207,), test_X=(63, 19)
   ✅ Datos filtrados: 204 filas válidas de 207 originales
   📊 Distribución de clases: {1: 111, 0: 93}
   ✅ Pipeline construido con 19 características
   ✅ Modelo primario entrenado exitosamente
   📈 Threshold óptimo: 0.500 (F1=1.000)
   🎯 Señales generadas: 15 compras, 48 ventas
   📊 Input data: train_X=(270, 19), train_y=(270,), test_X=(63, 19)
   ✅ Datos filtrados: 267 filas válidas de 270 originales
   📊 Distribución de clases: {1: 143, 0: 124}
   ✅ Pipeline construido con 19 características
   ✅ Modelo primario entrenado exitosamente
   📈 Threshold óptimo: 0.500 (F1=1.000)
   🎯 Señales generadas: 43 compras, 20 ventas
   📊 Input data: train_X=(333, 19), train_y=(333,), test_X=(63, 19)
   ✅ Datos filtrados: 319 filas válidas de 333 orig

In [14]:
# ====================================================================
# CELDA 4: ANÁLISIS DE SENSIBILIDAD DEL RIESGO (VERSIÓN DEFINITIVA)
# ====================================================================

import pandas as pd
import numpy as np
import json
import joblib
import vectorbt as vbt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

print("--- [INICIO] Análisis de Sensibilidad del Riesgo ---")

# --- 1. Cargar Artefactos ---
if 'X_final_auditado' not in locals() or 'strategy_package' not in locals():
    raise NameError("Por favor, ejecuta la CELDA 1 de este notebook primero.")

print(" -> Artefactos cargados correctamente.")
primary_model = strategy_package['primary_model_pipeline']
meta_model = strategy_package['meta_model']
optimal_threshold = strategy_package['optimal_threshold']

# --- 2. Generar Señales para Todo el Histórico ---
print(" -> Generando señales de trading para todo el histórico...")
all_tickers = X_final_auditado['ticker'].unique()
all_signals_list = []

for ticker in all_tickers:
    ticker_data = X_final_auditado[X_final_auditado['ticker'] == ticker].drop(columns=['ticker'])
    if ticker_data.empty: 
        continue
        
    primary_proba = primary_model.predict_proba(ticker_data)
    primary_preds = np.argmax(primary_proba, axis=1)
    meta_features = pd.DataFrame({'primary_model_prob': primary_proba.max(axis=1)}, index=ticker_data.index)
    meta_confidence = meta_model.predict_proba(meta_features)[:, 1]
    
    passes_threshold = meta_confidence >= optimal_threshold
    signals = np.zeros(len(ticker_data))
    signals[(primary_preds == 1) & passes_threshold] = 1
    signals[(primary_preds == 0) & passes_threshold] = -1
    all_signals_list.append(pd.Series(signals, index=ticker_data.index, name=ticker))

# REGENERAR las variables correctas (no usar las de prueba)
signals_df = pd.concat(all_signals_list, axis=1)
entries_real = signals_df == 1  # ✅ Usar datos reales
exits_real = signals_df == -1   # ✅ Usar datos reales

print("✅ Señales regeneradas correctamente.")
print(f"Signals shape: {signals_df.shape}")
print(f"Entries shape: {entries_real.shape}")
print(f"Exits shape: {exits_real.shape}")

# --- 3. Preparar Datos de Precios Reales ---
print(" -> Preparando datos de precios...")
close_prices_real = X_final_auditado.pivot(columns='ticker', values='close')  # ✅ Usar datos reales

print(f"Close prices shape: {close_prices_real.shape}")
print(f"Período de datos: {close_prices_real.index.min()} a {close_prices_real.index.max()}")

# Alinear todos los datos
common_index = close_prices_real.index.intersection(entries_real.index)
close_prices_real = close_prices_real.loc[common_index]
entries_real = entries_real.loc[common_index]
exits_real = exits_real.loc[common_index]

print(f"Datos alineados: {len(common_index)} fechas en común")
print(f"Total señales de entrada: {entries_real.sum().sum()}")
print(f"Total señales de salida: {exits_real.sum().sum()}")

# --- 4. Verificar que hay señales válidas ---
if entries_real.sum().sum() == 0:
    print("❌ ERROR: No hay señales de entrada válidas")
    raise ValueError("No se pueden ejecutar backtests sin señales de entrada")

if exits_real.sum().sum() == 0:
    print("❌ ERROR: No hay señales de salida válidas")
    raise ValueError("No se pueden ejecutar backtests sin señales de salida")

# --- 5. Ejecutar Backtest Parametrizado ---
print(" -> Ejecutando backtests para múltiples niveles de riesgo...")

risk_levels = np.arange(0.01, 0.105, 0.005)
results_list = []

for risk in tqdm(risk_levels, desc="Simulando Niveles de Riesgo"):
    try:
        portfolio = vbt.Portfolio.from_signals(
            close=close_prices_real,
            entries=entries_real,
            exits=exits_real,
            size=risk * 100,  # Convertir a porcentaje
            size_type='Percent',
            fees=0.002,
            sl_stop=0.05,
            init_cash=100000,
            freq='D'
        )
        
        # Obtener estadísticas con nombres correctos
        stats = portfolio.stats(['Total Return [%]', 'Max Drawdown [%]'])
        
        results_list.append({
            'risk_per_trade': risk,
            'Total Return': stats['Total Return [%]'],
            'Max Drawdown': stats['Max Drawdown [%]']
        })
        
    except Exception as e:
        print(f"Error con riesgo {risk:.3f}: {e}")
        continue

# Convertir resultados a DataFrame
results = pd.DataFrame(results_list)

print("✅ Simulaciones completadas.")
print(f"✅ Completadas {len(results)} de {len(risk_levels)} simulaciones.")

if results.empty:
    print("❌ ERROR: No se pudieron completar las simulaciones")
    print("Revisando problema específico...")
    
    # Diagnóstico específico del primer ticker
    first_ticker = close_prices_real.columns[0]
    print(f"Diagnosticando ticker: {first_ticker}")
    print(f"  Entradas: {entries_real[first_ticker].sum()}")
    print(f"  Salidas: {exits_real[first_ticker].sum()}")
    print(f"  Precios: {close_prices_real[first_ticker].dropna().shape[0]} días")
    
else:
    print("\n--- Resultados del Análisis de Sensibilidad ---")
    print(results)

    # --- 6. Análisis de Resultados ---
    # Encontrar configuración óptima
    results['risk_adjusted_return'] = results['Total Return'] / (results['Max Drawdown'] + 0.01)  # +0.01 para evitar división por 0
    best_config = results.loc[results['risk_adjusted_return'].idxmax()]
    
    print(f"\n--- Configuración Óptima ---")
    print(f"Riesgo por operación: {best_config['risk_per_trade']:.3f} ({best_config['risk_per_trade']*100:.1f}%)")
    print(f"Retorno total: {best_config['Total Return']:.2f}%")
    print(f"Máximo drawdown: {best_config['Max Drawdown']:.2f}%")
    print(f"Ratio riesgo-ajustado: {best_config['risk_adjusted_return']:.2f}")

    # --- 7. Visualización ---
    print("\n -> Generando gráficos...")
    
    plt.style.use('default')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Gráfico 1: Scatter plot
    results['risk_per_trade_pct'] = results['risk_per_trade'] * 100
    scatter = ax1.scatter(results['Max Drawdown'], results['Total Return'], 
                         s=100, c=results['risk_per_trade_pct'], 
                         cmap='viridis', alpha=0.7)
    
    # Marcar punto óptimo
    ax1.scatter(best_config['Max Drawdown'], best_config['Total Return'], 
               s=200, c='red', marker='*', edgecolor='black', linewidth=2, 
               label=f'Óptimo ({best_config["risk_per_trade"]*100:.1f}%)')
    
    ax1.set_title('Análisis de Sensibilidad: Drawdown vs. Retorno', fontsize=14)
    ax1.set_xlabel('Máximo Drawdown (%)', fontsize=12)
    ax1.set_ylabel('Retorno Total (%)', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    cbar = plt.colorbar(scatter, ax=ax1)
    cbar.set_label('Riesgo por Operación (%)', fontsize=10)

    # Gráfico 2: Líneas de tendencia
    ax2.plot(results['risk_per_trade_pct'], results['Total Return'], 
             'o-', label='Retorno Total', linewidth=2, markersize=6)
    ax2.plot(results['risk_per_trade_pct'], results['Max Drawdown'], 
             's-', label='Max Drawdown', linewidth=2, markersize=6)
    
    # Marcar punto óptimo
    ax2.axvline(best_config['risk_per_trade']*100, color='red', linestyle='--', 
               alpha=0.7, label=f'Óptimo ({best_config["risk_per_trade"]*100:.1f}%)')
    
    ax2.set_title('Métricas vs. Nivel de Riesgo', fontsize=14)
    ax2.set_xlabel('Riesgo por Operación (%)', fontsize=12)
    ax2.set_ylabel('Valor (%)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # --- 8. Recomendaciones Finales ---
    print("\n--- RECOMENDACIONES FINALES ---")
    
    returns_std = results['Total Return'].std()
    drawdown_std = results['Max Drawdown'].std()
    
    print(f"Volatilidad de retornos: {returns_std:.2f}%")
    print(f"Volatilidad de drawdowns: {drawdown_std:.2f}%")
    
    # Recomendaciones por perfil de riesgo
    low_dd_mask = results['Max Drawdown'] <= results['Max Drawdown'].quantile(0.3)
    if low_dd_mask.any():
        conservative_idx = results[low_dd_mask]['Total Return'].idxmax()
        conservative_config = results.loc[conservative_idx]
        print(f"\n📊 PERFIL CONSERVADOR:")
        print(f"  Riesgo: {conservative_config['risk_per_trade']*100:.1f}%")
        print(f"  Retorno: {conservative_config['Total Return']:.2f}%")
        print(f"  Drawdown: {conservative_config['Max Drawdown']:.2f}%")
    
    high_return_mask = results['Total Return'] >= results['Total Return'].quantile(0.7)
    if high_return_mask.any():
        aggressive_idx = results[high_return_mask]['Max Drawdown'].idxmin()
        aggressive_config = results.loc[aggressive_idx]
        print(f"\n🚀 PERFIL AGRESIVO:")
        print(f"  Riesgo: {aggressive_config['risk_per_trade']*100:.1f}%")
        print(f"  Retorno: {aggressive_config['Total Return']:.2f}%")
        print(f"  Drawdown: {aggressive_config['Max Drawdown']:.2f}%")

print("\n--- [FIN] Análisis de Sensibilidad del Riesgo ---")

--- [INICIO] Análisis de Sensibilidad del Riesgo ---
 -> Artefactos cargados correctamente.
 -> Generando señales de trading para todo el histórico...
✅ Señales regeneradas correctamente.
Signals shape: (459, 12)
Entries shape: (459, 12)
Exits shape: (459, 12)
 -> Preparando datos de precios...
Close prices shape: (459, 12)
Período de datos: 2021-01-03 00:00:00+00:00 a 2022-04-06 00:00:00+00:00
Datos alineados: 459 fechas en común
Total señales de entrada: 58
Total señales de salida: 3
 -> Ejecutando backtests para múltiples niveles de riesgo...


Simulando Niveles de Riesgo:   0%|          | 0/19 [00:00<?, ?it/s]

Error con riesgo 0.010: 'Total Return [%]'
Error con riesgo 0.015: 'Total Return [%]'
Error con riesgo 0.020: 'Total Return [%]'
Error con riesgo 0.025: 'Total Return [%]'
Error con riesgo 0.030: 'Total Return [%]'
Error con riesgo 0.035: 'Total Return [%]'
Error con riesgo 0.040: 'Total Return [%]'
Error con riesgo 0.045: 'Total Return [%]'
Error con riesgo 0.050: 'Total Return [%]'
Error con riesgo 0.055: 'Total Return [%]'
Error con riesgo 0.060: 'Total Return [%]'
Error con riesgo 0.065: 'Total Return [%]'
Error con riesgo 0.070: 'Total Return [%]'
Error con riesgo 0.075: 'Total Return [%]'
Error con riesgo 0.080: 'Total Return [%]'
Error con riesgo 0.085: 'Total Return [%]'
Error con riesgo 0.090: 'Total Return [%]'
Error con riesgo 0.095: 'Total Return [%]'
Error con riesgo 0.100: 'Total Return [%]'
✅ Simulaciones completadas.
✅ Completadas 0 de 19 simulaciones.
❌ ERROR: No se pudieron completar las simulaciones
Revisando problema específico...
Diagnosticando ticker: ADA-USD
  En